Author: Aydin Najl Hossaini  
Date: 24/08/2024

In [1]:
# Imports
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType, Team
from tqdm.notebook import  tqdm
import requests
from bs4 import BeautifulSoup

In [2]:
# Dictionary of team names from bball reference to the web scraper API team names
dict_teams = {"ATL" : Team.ATLANTA_HAWKS, "BOS" : Team.BOSTON_CELTICS, "BRK" : Team.BROOKLYN_NETS, "CHI" : Team.CHICAGO_BULLS, "CHO" : Team.CHARLOTTE_HORNETS, "CLE" : Team.CLEVELAND_CAVALIERS,
"DAL" : Team.DALLAS_MAVERICKS, "DEN" : Team.DENVER_NUGGETS, "DET" : Team.DETROIT_PISTONS, "GSW" : Team.GOLDEN_STATE_WARRIORS, "HOU" : Team.HOUSTON_ROCKETS, "IND" : Team.INDIANA_PACERS, "LAC" : Team.LOS_ANGELES_CLIPPERS, "LAL" : Team.LOS_ANGELES_LAKERS, "MEM" : Team.MEMPHIS_GRIZZLIES,
"MIA" : Team.MIAMI_HEAT, "MIL" : Team.MILWAUKEE_BUCKS, "MIN" : Team.MINNESOTA_TIMBERWOLVES, "NOP" : Team.NEW_ORLEANS_PELICANS, "NYK" : Team.NEW_YORK_KNICKS, "OKC" : Team.OKLAHOMA_CITY_THUNDER,
"ORL" : Team.ORLANDO_MAGIC, "PHI" : Team.PHILADELPHIA_76ERS, "PHO" : Team.PHOENIX_SUNS, "POR" : Team.PORTLAND_TRAIL_BLAZERS, "SAC" : Team.SACRAMENTO_KINGS, "SAS" : Team.SAN_ANTONIO_SPURS,
"TOR" : Team.TORONTO_RAPTORS, "UTA" : Team.UTAH_JAZZ, "WAS" : Team.WASHINGTON_WIZARDS}

def get_table_info(rows : list) -> list:
    """ Extracts the date, opponent, and home/away status from given rows """
    
    game_data = []
    for row in rows:
        if 'thead' in row.get('class', []):  # Skip header rows
            continue

        # Extract the date, opponent, and home/away status
        date = row.find('td', {'data-stat': 'date_game'}).text
        opponent = row.find('td', {'data-stat': 'opp_id'}).text
        home_game = 'Home' if not row.find('td', {'data-stat': 'game_location'}).contents else 'Away' # game_location column is empty for home games but has "@" symbol for away games, check if the field is empty to determine home/away status.

        # Append to the list
        game_data.append([date, home_game, opponent])

    return game_data

# Get all the games of Lebron in the 2018 season
url = "https://www.basketball-reference.com/players/j/jamesle01/gamelog/2018/"
soup = BeautifulSoup(requests.get(url).content, "html.parser") # Gets full page from URL and parses it with HTML parser

if soup is None:
    raise Exception("Failed to load page, check URL")
else: 
    table = soup.find('table', {'id': 'pgl_basic'})  # Find table by ID
    rows = table.find('tbody').find_all('tr') # Find all rows in the table

    game_data = get_table_info(rows)

    for row in game_data:
        row[0] = [int(i) for i in row[0].split("-")] # split the string into day, month, year and convert to integers and keep in the same list

game_data[:3]
        

[[[2017, 10, 17], 'Home', 'BOS'],
 [[2017, 10, 20], 'Away', 'MIL'],
 [[2017, 10, 21], 'Home', 'ORL']]

In [64]:
# Output all advanced player season totals for the 2017-2018 season in CSV format to 2018_10_06_BOS_PBP.csv
def scrape_games(game_data : list) -> list:
    """ Scrapes the play by play data for each game in the game_data list with the format [[year, month, day], home/away, opponent]
    and stores it in a CSV file in the pbp_games folder """

    for game in game_data:
        year, month, day = game[0]

        print(f"Writing play-by-play for Cavs game on {year}-{month}-{day} to CSV file")
        try: # Stores all PBP as CSV's in folder pbp_games
            if game[1] == "Home":
                client.play_by_play(home_team=Team.CLEVELAND_CAVALIERS, year=year, month=month, day=day, output_type=OutputType.CSV, output_file_path=f"pbp_games/{year}_{month}_{day}_CLE_PBP_HOME.csv")
            elif game[1] == "Away":
                client.play_by_play(home_team=dict_teams[game[2]], year=year, month=month, day=day, output_type=OutputType.CSV, output_file_path=f"pbp_games/{year}_{month}_{day}_CLE_PBP_AWAY.csv")
            else:
                print("Error in home/away")
        except Exception:
            print("Failed play by play")
            
# scrape_games(game_data)

Writing play-by-play for Cavs game on 2017-10-17 to CSV file
Writing play-by-play for Cavs game on 2017-10-20 to CSV file
Writing play-by-play for Cavs game on 2017-10-21 to CSV file
Writing play-by-play for Cavs game on 2017-10-24 to CSV file
Writing play-by-play for Cavs game on 2017-10-25 to CSV file
Writing play-by-play for Cavs game on 2017-10-28 to CSV file
Writing play-by-play for Cavs game on 2017-10-29 to CSV file
Writing play-by-play for Cavs game on 2017-11-1 to CSV file
Writing play-by-play for Cavs game on 2017-11-3 to CSV file
Writing play-by-play for Cavs game on 2017-11-5 to CSV file
Writing play-by-play for Cavs game on 2017-11-7 to CSV file
Writing play-by-play for Cavs game on 2017-11-9 to CSV file
Writing play-by-play for Cavs game on 2017-11-11 to CSV file
Writing play-by-play for Cavs game on 2017-11-13 to CSV file
Writing play-by-play for Cavs game on 2017-11-15 to CSV file
Writing play-by-play for Cavs game on 2017-11-17 to CSV file
Writing play-by-play for Cavs

TODO  
1. Change teamnname in the home_team in the away games (get the away name from the same page (check if @=True then away)) DONE
2. Add testing for scraper
3. Cut csv to contain only LBJ free throws 
    Maybe in the future use FG's as well 
4. Use prediction model


In [70]:
# Filter the useful information from the play by play data
import pandas as pd

game = pd.read_csv("pbp_games/2017_10_17_CLE_PBP_HOME.csv")
game.headb

period                           int64
period_type                     object
remaining_seconds_in_period    float64
relevant_team                   object
away_team                       object
home_team                       object
away_score                       int64
home_score                       int64
description                     object
dtype: object

In [71]:
game.head()
# Transfer period type + remaining_seconds_in_period to a time format
# Full quarter has 720 seconds, therefore 720 - remaining_seconds_in_period = seconds played in the quarter
# If quarter is 2, 2*720 - remaining_seconds_in_period ;  3 + 3*720 - ..., 4 + 4*720 - ... 

,period,period_type,remaining_seconds_in_period,relevant_team,away_team,home_team,away_score,home_score,description
0,1,QUARTER,704.0,BOSTON CELTICS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,K. Irving makes 2-pt jump shot from 10 ft (ass...
1,1,QUARTER,687.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,D. Rose misses 2-pt layup from 1 ft (block by ...
2,1,QUARTER,683.0,BOSTON CELTICS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,Defensive rebound by A. Horford
3,1,QUARTER,681.0,BOSTON CELTICS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,G. Hayward misses 3-pt jump shot from 25 ft
4,1,QUARTER,678.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,Defensive rebound by D. Rose


# Decide on features
 1. Time remaining in the quarter
 2. Score difference
 3. Home/Away team
 4. Period
 5. how many made shots in the game (=FG%)  
     Obtain this through the description column and filter for "makes" and "misses" on player level
        Notable words: 
        misses 2-pt
        misses 3-pt
        misses free throw
        makes free throw
        makes 2-pt
        makes 3-pt


In [76]:
game[game["description"].str.contains("L. James")] # 1. Lebron made 12 shots in the game

,period,period_type,remaining_seconds_in_period,relevant_team,away_team,home_team,away_score,home_score,description
7,1,QUARTER,651.0,BOSTON CELTICS,BOSTON CELTICS,CLEVELAND CAVALIERS,2,0,J. Tatum misses 2-pt layup from 2 ft (block by...
15,1,QUARTER,589.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,4,5,D. Wade makes 2-pt jump shot from 12 ft (assis...
17,1,QUARTER,571.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,4,5,Defensive rebound by L. James
18,1,QUARTER,569.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,4,7,D. Rose makes 2-pt layup from 1 ft (assist by ...
40,1,QUARTER,426.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,10,9,Defensive rebound by L. James
...,...,...,...,...,...,...,...,...,...
439,4,QUARTER,144.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,96,95,L. James makes 2-pt dunk from 1 ft (assist by ...
448,4,QUARTER,79.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,98,99,L. James makes 2-pt layup from 2 ft
451,4,QUARTER,46.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,98,102,K. Love makes 3-pt jump shot from 22 ft (assis...
458,4,QUARTER,31.0,CLEVELAND CAVALIERS,BOSTON CELTICS,CLEVELAND CAVALIERS,99,102,Defensive rebound by L. James
